In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

from fbprophet import Prophet
from datetime import date
from datetime import datetime
from datetime import timedelta

import glob

from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
import itertools

from azureml.core import Workspace,Datastore, Dataset

pd.options.mode.chained_assignment = None  # default='warn'


def load_data(data_file):
    ###
    #### For anon data, load_data is the only function needed to change
    #### + Update IDs in model_config
    ###
    
    ### Pull from Blob
    ws = Workspace.from_config()
    blob_datastore_name='cesmii_datastore'
    datastore = Datastore.get(ws, datastore_name=blob_datastore_name)
    datastore_paths = [(datastore, data_file)]
    ds = Dataset.Tabular.from_delimited_files(path=datastore_paths, header='ALL_FILES_HAVE_SAME_HEADERS')
    data = ds.to_pandas_dataframe()
    ###

    data = data.loc[data["Transaction Type"] == "BK"]
    data['ID'] = (data["Plant Code"] + '_' + data["Part Number"].astype(str))
    data.sort_values(by = 'ID',inplace = True)
    parts = list(data["ID"].unique())

    data.index = pd.to_datetime(data["Inv Adjustment Datetime"])
    data_sorted = data.sort_index()

    present = data_sorted.index.max().date().strftime("%m/%d/%Y")

    return parts, data_sorted, present

def get_raw_data(data,
                 in_ID, 
                 outlier_cutoff = ''):

    data_sorted = data

    print(in_ID)
    raw_data = -1*data_sorted.loc[(data_sorted.ID == in_ID),
                                   "Transaction Qty"]
    if outlier_cutoff != '' :
        raw_data = raw_data.loc[raw_data < outlier_cutoff]
        
    return raw_data



def get_integrated_time( raw_data,
                         integral_window,
                         history_start_date,
                         center_date,
                         history_end_date,
                         evaluation_frequency ):
    
    delta =  timedelta(days = (
                                (pd.to_datetime(center_date) - pd.to_datetime(history_start_date)).days) %
                                int(evaluation_frequency[0])
                              )

    start_dt = pd.to_datetime(history_start_date) + delta 

    date_grid = pd.date_range(
                              start= start_dt, 
                              end= history_end_date, 
                              freq= evaluation_frequency, 
                             )

    value_out = []

    for date in date_grid:
        left_window = date - timedelta(days= integral_window)
        value_out.append(raw_data[left_window:date].sum())

    integral = pd.DataFrame(value_out)
    integral.index = date_grid

    integral = integral[date_grid[0] + timedelta(days= integral_window):]
    
    return integral



def cv_model_tune(processed_data):

    df = pd.DataFrame()
    df['ds'] = processed_data.index
    df['y'] = processed_data.values

    param_grid = {
        'daily_seasonality' : [False],
        'weekly_seasonality' : [False],
        'yearly_seasonality' : [True],
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial='366 days',period = '15 days', horizon='365 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses

    [m_daily_seasonality,
     m_weekly_seasonality,
     m_yearly_seasonality,
     m_changepoint_prior_scale,
     m_seasonality_prior_scale,
     rmse] = tuning_results.loc[tuning_results.rmse == tuning_results.rmse.min()].values[0]
    
    return m_daily_seasonality, m_weekly_seasonality, m_yearly_seasonality, m_changepoint_prior_scale, m_seasonality_prior_scale, rmse



def get_model_fit_line(processed_data,
                       params):

    dff = processed_data
    data = pd.DataFrame()
    data['ds'] = dff.index
    data['y'] = dff.values

    m = Prophet( 
                  weekly_seasonality= params['weekly_seasonality'],
                  daily_seasonality= params['daily_seasonality'],
                  changepoint_prior_scale =  params['changepoint_prior_scale'], 
                  changepoint_range = params['changepoint_range'], 
                  seasonality_prior_scale = params['seasonality_prior_scale'],  
                  yearly_seasonality = params['yearly_seasonality'],
                  interval_width=params['interval_width']
               )

    m.fit(data)

    future = m.make_future_dataframe(periods=1)
    future.tail()

    forecast = m.predict(future)

    fig1 = m.plot(forecast)



def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    idx = (y_true!=0)

    mape_score = np.mean(np.abs((y_true[idx] - y_pred[idx]) / y_true[idx]))

    return mape_score



def model_eval_forecast( processed_data,
                         raw_data,
                         center_date,
                         forecast_horizon,
                         params,
                         eval_every_N_weeks = 2
                       ):

    curve_in = processed_data
    every_N_weeks = eval_every_N_weeks

    relief_forecast_results_out = []

    results = pd.DataFrame(index = ['Max','Mean','Min','Std'])

    mape = []
    present = pd.to_datetime(center_date)
    for i in range(0,52):
        horizon = present + timedelta(days= forecast_horizon)

        dff_train = curve_in[:present]    
        dff_test = curve_in[present:horizon][1:]


        data = pd.DataFrame()
        data['ds'] = dff_train.index
        data['y'] = dff_train.values

        test = pd.DataFrame()
        test['ds'] = dff_test.index

        m = Prophet( 
                      yearly_seasonality=True,            
                      weekly_seasonality= False,
                      daily_seasonality= False,
                      changepoint_prior_scale = params['changepoint_prior_scale'],
                      seasonality_prior_scale = params['seasonality_prior_scale'],
                      changepoint_range = .8,
                   )



        m.fit(data)

        if dff_test.index.max() < curve_in.index.max():

             forecast = m.predict(test)

             mape.append(MAPE(dff_test[0],forecast.yhat))

             forecast = forecast.loc[pd.to_datetime(forecast.ds) == horizon]

             right_window = forecast.ds.values[0]
             left_window = present

             relief_forecast_results_out.append([
                                                    left_window,
                                                    right_window,
                                                    forecast.yhat_upper.values[0],
                                                    forecast.yhat.values[0],
                                                    forecast.yhat_lower.values[0],
                                                    raw_data[left_window:right_window].sum(),
                                                ])

        present = present + timedelta(weeks= every_N_weeks)

    MAPE_metric = pd.DataFrame(mape)
    Accuracy = (1 - MAPE_metric)*100
    temp = pd.DataFrame([Accuracy.max(),Accuracy.mean(),Accuracy.min(),Accuracy.std()],index = ['Max','Mean','Min','Std'])
    
    results[str(forecast_horizon) + 'days'] = temp

    BK_results = pd.DataFrame(relief_forecast_results_out, columns = ['now', 
                                                                      'forecast_date', 
                                                                      'High',
                                                                      'Mean',
                                                                      'Low',
                                                                      'Actual']
                              )

    BK_results['error'] = ((BK_results.Mean - BK_results.Actual)/BK_results.Actual)
    
    return [results, BK_results]



def model_forecast(processed_data,
                   forecast_horizon,
                   params,
                   evaluation_frequency):

    dff_train = processed_data

    data = pd.DataFrame()
    data['ds'] = dff_train.index
    data['y'] = dff_train.values

    future_index = pd.date_range(
                                  start= processed_data.index[-1], 
                                  end= processed_data.index[-1] + timedelta(days= forecast_horizon), 
                                  freq= evaluation_frequency 
                                 )

    forecast_index = processed_data.index

    eval_index = pd.DataFrame()
    eval_index['ds'] = forecast_index.append(future_index[1:])

    m = Prophet(yearly_seasonality=True,            
                weekly_seasonality= False,
                daily_seasonality= False,
                changepoint_prior_scale = params['changepoint_prior_scale'],
                seasonality_prior_scale = params['seasonality_prior_scale'],
                changepoint_range = .8,
               )

    m.fit(data)

    forecast = m.predict(eval_index)

    right_window = [forecast.iloc[-1].ds][0]
    left_window = [forecast.iloc[-1].ds][0] - timedelta(days= forecast_horizon)


    left_window,
    right_window,
    forecast_value = forecast.iloc[-1].yhat
    
    return left_window, right_window, forecast_value
    
    

def get_forecast_metrics(forecast_actuals):
    coverage = (sum((forecast_actuals.Actual < forecast_actuals.High) & (forecast_actuals.Actual > forecast_actuals.Low))/len(forecast_actuals))*100
    forecast_MAPE = forecast_actuals.error.abs().mean()*100
    forecast_MAPE_std = forecast_actuals.error.abs().std()*100
    forecast_mean_error = forecast_actuals.error.mean()*100
    forecast_std_error = forecast_actuals.error.std()*100
    
    return coverage, forecast_mean_error, forecast_std_error, forecast_MAPE, forecast_MAPE_std

def get_part_descriptors(in_ID, data):

    data['ID'] = (data["Plant Code"] + '_' + data["Part Number"].astype(str))
    
    data_sorted = data
    
    descriptor = data_sorted.loc[(data_sorted.ID == in_ID) &
                                 (data_sorted["Transaction Type"] == "BK"),
                                             ["Commodity Description",
                                              "Sub Commodity Description", 
                                              "Primary Supplier Name", 
                                              "Part Description"]].iloc[0]
        
    return list(descriptor.values)

##PART OF MAIN CALL##

def instance_eval(data, PRESENT, MODEL_CONFIG_FILE):

    model_config = pd.read_csv('./files/model_cvs/' + MODEL_CONFIG_FILE, low_memory=False, index_col=False).fillna('')

    center_date = (data.index.max() - timedelta(days= 364)).date().strftime("%m/%d/%Y")

    model_eval = pd.DataFrame([], columns = [   'part_ID', 
                                                'outlier_cutoff', 
                                                'start_date', 
                                                'center_date',
                                                'present', 
                                                'evaluation_frequency', 
                                                'forecast_window', 
                                                'model_eval_every_N_weeks',
                                                'm_changepoint_prior_scale',
                                                'm_seasonality_prior_scale',
                                                'goodness_fit_Max',
                                                'goodness_fit_Mean',
                                                'goodness_fit_Min',
                                                'goodness_fit_Std',
                                                'forecast_mean_error',
                                                'forecast_std_error',
                                                'forecast_MAPE',
                                                'forecast_MAPE_std',
                                                'N_test_points'
                                            ])

    for curve in model_config.iterrows():
        
        part_ID = curve[1].part_ID
        forecast_horizon = curve[1].forecast_window
        outlier_cutoff = curve[1].outlier_cutoff
        start_date = curve[1].start_date
        evaluation_frequency = curve[1].evaluation_frequency
        end_date = PRESENT
        model_eval_every_N_weeks = 1
        
        
        raw_data = get_raw_data(
                                data,
                                part_ID, 
                                outlier_cutoff
                            )

        processed_data = get_integrated_time( raw_data,
                                            forecast_horizon,
                                            start_date,
                                            center_date,
                                            end_date,
                                            evaluation_frequency )

        parameters = { 
                    'changepoint_prior_scale' : curve[1].m_changepoint_prior_scale,  
                    'seasonality_prior_scale': curve[1].m_seasonality_prior_scale,
                    }

        [goodness_fit, forecast_actuals] = model_eval_forecast(  
                                                                processed_data,
                                                                raw_data,
                                                                center_date,
                                                                forecast_horizon ,
                                                                params = parameters,
                                                                eval_every_N_weeks = model_eval_every_N_weeks
                                                            )

        _, mean_error, std_error, MAPE_score, MAPE_std = get_forecast_metrics(forecast_actuals)

        temp = pd.DataFrame(   [part_ID, 
                                outlier_cutoff, 
                                start_date, 
                                center_date, 
                                end_date, 
                                evaluation_frequency, 
                                forecast_horizon, 
                                model_eval_every_N_weeks,
                                parameters['changepoint_prior_scale'],
                                parameters['seasonality_prior_scale'],
                                goodness_fit.loc['Max'].values[0],
                                goodness_fit.loc['Mean'].values[0],
                                goodness_fit.loc['Min'].values[0],
                                goodness_fit.loc['Std'].values[0],
                                mean_error, 
                                std_error,
                                MAPE_score,
                                MAPE_std,
                                len(forecast_actuals)]

                            ).transpose()

        temp.columns = [    'part_ID', 
                            'outlier_cutoff', 
                            'start_date', 
                            'center_date',
                            'present', 
                            'evaluation_frequency', 
                            'forecast_window', 
                            'model_eval_every_N_weeks',
                            'm_changepoint_prior_scale',
                            'm_seasonality_prior_scale',
                            'goodness_fit_Max',
                            'goodness_fit_Mean',
                            'goodness_fit_Min',
                            'goodness_fit_Std',
                            'forecast_mean_error',
                            'forecast_std_error',
                            'forecast_MAPE',
                            'forecast_MAPE_std',
                            'N_test_points'
                        ]

        model_eval = model_eval.append(temp)

    return model_eval

def forecast_instance(data, model_eval):

    forecasts = pd.DataFrame([], columns = [ 'part_ID', 
                                            'present', 
                                            'forecast_window',
                                            'forecast_date',
                                            'forecast_value',
                                            'historical_mean_error',
                                            'historical_error_std',
                                            'forecast_MAPE',
                                            'accuracy',
                                            'forecast_low',
                                            'forecast_high',
                                            'Commodity Description',
                                            'Sub Commodity Description', 
                                            'Primary Supplier Name', 
                                            'Part Description']
                            )

    z = 1.44 #85% percentile
    forecast_present = model_eval.present.iloc[0]

    for instance in model_eval.iterrows():     

        in_ID = instance[1].part_ID
        outlier_cutoff = instance[1].outlier_cutoff
        forecast_window = int(instance[1].forecast_window)
        start_date = instance[1].start_date
        evaluation_frequency = instance[1].evaluation_frequency

        center_date = forecast_present
        end_date = forecast_present

        parameters = { 
                    'changepoint_prior_scale' : instance[1].m_changepoint_prior_scale, 
                    'seasonality_prior_scale': instance[1].m_seasonality_prior_scale,  
                    }

        raw_data = get_raw_data(
                                data,
                                in_ID, 
                                outlier_cutoff
                            )

        processed_data = get_integrated_time( raw_data,                                     
                                            forecast_window,
                                            start_date,
                                            center_date,
                                            end_date,
                                            evaluation_frequency
                                            )

        present, forecast_date, forecast_value = model_forecast(
                                                                processed_data,
                                                                forecast_window,
                                                                parameters,
                                                                evaluation_frequency
                                                            )

        output = [[
                    in_ID, 
                    present, 
                    forecast_window, 
                    forecast_date, 
                    int(forecast_value), 
                    round(instance[1].forecast_mean_error,2),
                    round(instance[1].forecast_std_error,2),
                    round(instance[1].forecast_MAPE,2),
                    round(100 - instance[1].forecast_MAPE,2),
                    max(int(forecast_value*(1 - (z*instance[1].forecast_std_error/100))),0),
                    int(forecast_value*(1 + (z*instance[1].forecast_std_error/100)))]
                    + get_part_descriptors(in_ID, data)
                ]

        temp = pd.DataFrame(output, columns = ['part_ID', 
                                            'present', 
                                            'forecast_window',
                                            'forecast_date',
                                            'forecast_value',
                                            'historical_mean_error',
                                            'historical_error_std',
                                            'forecast_MAPE',
                                            'accuracy',
                                            'forecast_low',
                                            'forecast_high',
                                            'Commodity Description',
                                            'Sub Commodity Description', 
                                            'Primary Supplier Name', 
                                            'Part Description']
                            )
        

        forecasts = forecasts.append(temp)

    return forecasts[['part_ID',
                      'present',
                      'forecast_window',
                      'forecast_date',
                      'forecast_low',
                      'forecast_value',
                      'forecast_high',
                      'accuracy',
                      'Commodity Description',
                      'Sub Commodity Description', 
                      'Primary Supplier Name', 
                      'Part Description']]

def check_for_update(PRESENT):
    files = glob.glob('./files/model_evals/model_eval_*')

    if len(files) > 0:
        chk = 1
        for csv in files:
            date = re.sub(r'_', r'/',csv.split('eval_')[-1][0:-4])
            chk = chk*(pd.to_datetime(PRESENT) >= pd.to_datetime(date))
    else:
        chk = 0
        
    return chk == 0

